In [1]:
import pandas as pd
import json
df = pd.read_csv('ali-baba-and-forty-thieves.actions_args.csv')

with open('ali-baba-and-forty-thieves.srl.json') as json_data:
    srl = json.load(json_data)
sentences_df = pd.read_csv('ali-baba-and-forty-thieves.sentences.csv')
sentences = sentences_df['text'].to_list()
sentences

['The story takes place in Baghdad during the Abbasid era. ',
 'Ali Baba and his elder brother Cassim are the sons of a merchant. ',
 "After the death of their father, the greedy Cassim marries a wealthy woman and becomes well-to-do, building on their father's business - but Ali Baba marries a poor woman and settles into the trade of a woodcutter. ",
 'One day Ali Baba is at work collecting and cutting firewood in the forest, and he happens to overhear a group of forty thieves visiting their treasure store. ',
 'The treasure is in a cave, the mouth of which is sealed by magic. ',
 'It opens on the words "Open, Simsim", and seals itself on the words "Close, Simsim". ',
 'When the thieves are gone, Ali Baba enters the cave himself, and takes some of the treasure home. ',
 "Ali Baba borrows his sister-in-law's scales to weigh this new wealth of gold coins. ",
 'Unbeknownst to Ali, she puts a blob of wax in the scales to find out what Ali is using them for, as she is curious to know what k

In [6]:
def get_additional_arguments(df, srl, sentences):
    adobj_start_bytes = []
    adobj_end_bytes = []
    offsets = []
    for idx in range(df.shape[0]):
        row = df.iloc[idx]
        s_id, v_id = row['sentence_id'], row['verb_id']
        token_start_end_bytes = get_tokens_start_end_bytes(srl[s_id]['words'], sentences[s_id])
        sentence, verb = sentences[s_id], srl[s_id]['verbs'][v_id]
        dobj_end = row['dobj_end_byte']
        tags = verb['tags']
        adobj_start_byte, adobj_end_byte = None, None

        start_fixed = False
        for i, tag in enumerate(tags):
            # if it's a modifier argument after the verb, we add modifier argument columns
            if check_M_arg(tag) and token_start_end_bytes[i]['start_byte'] > dobj_end:
                if not start_fixed:
                    adobj_start_byte, adobj_end_byte = token_start_end_bytes[i]['start_byte'], token_start_end_bytes[i][
                        'end_byte']
                    start_fixed = True
                else:
                    adobj_end_byte = token_start_end_bytes[i]['end_byte']

                j = i + 1
                while check_M_arg(tags[j]):
                    adobj_end_byte = token_start_end_bytes[j]['end_byte']
                    j += 1

        adobj_start_bytes.append(adobj_start_byte)
        adobj_end_bytes.append(adobj_end_byte)
        if adobj_start_byte:
            offsets.append(row['subj_start_byte_text'] - row['subj_start_byte'])
        else:
            offsets.append(None)

    df['adobj_start_bytes'] = adobj_start_bytes
    df['adobj_end_bytes'] = adobj_end_bytes
    df['adobj_start_bytes_text'] = [adobj_start_bytes[i] + offsets[i] if offsets[i] else None for i in
                                    range(len(offsets))]
    df['adobj_end_bytes_text'] = [adobj_end_bytes[i] + offsets[i] if offsets[i] else None for i in range(len(offsets))]
    return df
    

In [7]:
get_additional_arguments(df, srl, sentences)

,sentence_id,verb_id,verb,verb_start_byte,verb_end_byte,verb_start_byte_text,verb_end_byte_text,subj_coref_ids,subj_start_byte,subj_end_byte,...,dobj_start_byte,dobj_end_byte,dobj_start_byte_text,dobj_end_byte_text,supersense_category,event_label,adobj_start_bytes,adobj_end_bytes,adobj_start_bytes_text,adobj_end_bytes_text
0,2,0,marries,51,58,174,181,2,33.0,50.0,...,59.0,74.0,182.0,197.0,verb.social,1,NaN,NaN,NaN,NaN
1,2,1,becomes,79,86,202,209,2,33.0,50.0,...,87.0,97.0,210.0,220.0,verb.change,1,NaN,NaN,NaN,NaN
2,2,2,building,99,107,222,230,2,33.0,50.0,...,108.0,134.0,231.0,257.0,verb.consumption,1,NaN,NaN,NaN,NaN
3,2,3,marries,150,157,273,280,1,141.0,149.0,...,158.0,170.0,281.0,293.0,verb.social,1,NaN,NaN,NaN,NaN
4,3,1,collecting,28,38,366,376,1,8.0,16.0,...,51.0,59.0,389.0,397.0,verb.contact,1,60.0,73.0,398.0,411.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,37,2,tried,74,79,4571,4576,55,64.0,73.0,...,80.0,91.0,4577.0,4588.0,verb.social,1,NaN,NaN,NaN,NaN
79,37,3,kill,83,87,4580,4584,55,64.0,73.0,...,88.0,91.0,4585.0,4588.0,verb.contact,1,NaN,NaN,NaN,NaN
80,37,4,gives,96,101,4593,4598,1,93.0,95.0,...,102.0,110.0,4599.0,4607.0,verb.possession,1,NaN,NaN,NaN,NaN
81,37,5,marries,127,134,4624,4631,1,93.0,95.0,...,135.0,138.0,4632.0,4635.0,verb.social,1,139.0,149.0,4636.0,4646.0


In [ ]:
adobj_start_bytes = []
adobj_end_bytes = []
offsets = []
for idx in range(df.shape[0]):
    row = df.iloc[idx]
    s_id,v_id = row['sentence_id'], row['verb_id']
    token_start_end_bytes = get_tokens_start_end_bytes(srl[s_id]['words'], sentences[s_id])
    sentence, verb = sentences[s_id], srl[s_id]['verbs'][v_id]
    dobj_end = row['dobj_end_byte']
    tags = verb['tags']
    adobj_start_byte, adobj_end_byte = None,None
    
    start_fixed = False
    for i, tag in enumerate(tags):        
        #if it's a modifier argument after the verb, we add modifier argument columns
        if check_M_arg(tag) and token_start_end_bytes[i]['start_byte']>dobj_end:
            if not start_fixed:
                adobj_start_byte, adobj_end_byte = token_start_end_bytes[i]['start_byte'], token_start_end_bytes[i]['end_byte']
                start_fixed = True
            else:
                adobj_end_byte = token_start_end_bytes[i]['end_byte']

            j=i+1
            while check_M_arg(tags[j]):
                adobj_end_byte = token_start_end_bytes[j]['end_byte']
                j+=1
                
    adobj_start_bytes.append(adobj_start_byte)
    adobj_end_bytes.append(adobj_end_byte)
    if adobj_start_byte:
        offsets.append(row['subj_start_byte_text']-row['subj_start_byte'])
    else:
        offsets.append(None)
            
                
df['adobj_start_bytes'] = adobj_start_bytes
df['adobj_end_bytes'] = adobj_end_bytes
df['adobj_start_bytes_text'] = [adobj_start_bytes[i]+offsets[i] if offsets[i] else None for i in range(len(offsets))]
df['adobj_end_bytes_text'] = [adobj_end_bytes[i]+offsets[i] if offsets[i] else None for i in range(len(offsets))]

            
            
            
#             arg_token_start_end_bytes.append((tokens_start_end_bytes[i]['start_byte'], tokens_start_end_bytes[i]['end_byte']))
#             tokens.append(words[i])
#             j = i + 1
#             while (j < len(verb['tags'])) and check_I_arg(verb['tags'][j]):
#                 tokens.append(words[j])
#                 arg_token_start_end_bytes.append((tokens_start_end_bytes[j]['start_byte'], tokens_start_end_bytes[j]['end_byte']))
#                 j += 1
#             # Get start and end bytes of tokens
#             start_byte = tokens_start_end_bytes[i]['start_byte']
#             end_byte = tokens_start_end_bytes[j-1]['end_byte']
    
#     print(token_start_end_bytes)

In [ ]:
df

In [4]:
import re
# SRL Argument Functions
def check_B_arg(text: str):
    return re.search(r'B-ARG\d+', text) is not None

def check_I_arg(text: str):
    return re.search(r'I-ARG\d+', text) is not None

def check_arg(text: str):
    return re.search(r'ARG\d+', text) is not None

def check_verb(text: str):
    return re.search(r'\w-V', text) is not None

def check_M_arg(text: str):
    return re.search(r'\D-ARGM\D', text)

In [ ]:
check_B_arg(sentence[0])

In [ ]:
sentence[0]

In [ ]:
re.search(r'\D-ARGM\D', srl[0]['verbs'][0]['tags'][4])

In [ ]:
srl[0]['verbs'][0]['tags']

In [ ]:
srl[0]

In [5]:

def get_tokens_start_end_bytes(tokens, sentence):
    token_start_end_bytes_sentence = []

    s = 0
    for i, token in enumerate(tokens):
        token_start_end_bytes = {}

        token_start_end_bytes['token'] = token
        token_start_end_bytes['i'] = i

        start_byte = s
        end_byte = s + len(token)

        if sentence[start_byte] == ' ':
            start_byte += 1
            end_byte += 1

        #print(start_byte, end_byte)
        #print(sentence[start_byte])
        #print(sentence[end_byte])
        #print("mapping :", token, sentence[start_byte:end_byte])
        #print("\n")

        if token != sentence[start_byte:end_byte]:
            print('Incorrect mapping of token to sentence bytes.')
            print(i, token, sentence[start_byte:end_byte])

        token_start_end_bytes['start_byte'] = start_byte
        token_start_end_bytes['end_byte'] = end_byte

        token_start_end_bytes_sentence.append(token_start_end_bytes)

        s = end_byte

    return token_start_end_bytes_sentence